In [1]:
import numpy as np
import pandas as pd

# Load positive tweets
pos_tweets = []
with open('twitter-datasets/train_pos_full.txt', encoding = 'utf-8') as f:
    for line in f:
        pos_tweets.append(line)
        
# Load negative tweets
neg_tweets = []
with open('twitter-datasets/train_neg_full.txt', encoding = 'utf-8') as f:
    for line in f:
        neg_tweets.append(line)

In [2]:
all_tweets = np.array(pos_tweets + neg_tweets)

In [3]:
y = np.concatenate((np.ones(len(pos_tweets)), np.zeros(len(neg_tweets))))

random_idxs = np.random.permutation(len(y))

all_tweets = all_tweets[random_idxs]

y = y[random_idxs]

N_train = int(0.8*len(y))

train, test = all_tweets[:N_train], all_tweets[N_train:]
y_train, y_test = y[:N_train], y[N_train:]

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer 
 
tfidf_vectorizer=TfidfVectorizer(use_idf=True, ngram_range = (1,2)) 
X_train = tfidf_vectorizer.fit_transform(train)
X_test = tfidf_vectorizer.transform(test)

In [5]:
first_vector_tfidfvectorizer=X_train[0] 
df = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)


,tfidf
redknapp is,0.362565
user redknapp,0.362565
england job,0.345602
redknapp,0.318717
the england,0.318717
...,...
from camara,0.000000
from calvin,0.000000
from caloocan,0.000000
from calne,0.000000


In [6]:
from sklearn import linear_model

clf =  linear_model.SGDClassifier(loss = 'log', max_iter=int(1e7), tol=1e-5, verbose = False)


clf.fit(X_train, y_train)
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_test) == y_test).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

Training set accuracy: 76.37% / validation set: 76.53%


In [11]:
from sklearn.svm import LinearSVC

clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = False, C = 0.03)
clf.fit(X_train, y_train)
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_test) == y_test).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

Training set accuracy: 86.11% / validation set: 84.37%


In [8]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-6,
                    hidden_layer_sizes=(10, 5), random_state=0, verbose = True, max_iter = 1000)

clf.fit(X_train, y_train)
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_test) == y_test).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

KeyboardInterrupt: 

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=1000, n_iter=7, random_state=42)

X_train_ = svd.fit_transform(X_train)
X_test_ = svd.transform(X_test)


clf = LinearSVC(random_state=0, tol=1e-9, loss = 'squared_hinge', dual = False, C = 0.1)
clf.fit(X_train_, y_train)
train_acc = (clf.predict(X_train_) == y_train).mean()
val_acc = (clf.predict(X_test_) == y_test).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)  
clf.fit(X_train, y_train)


predictions = clf.predict(X_test)

In [ ]:
train_acc = (clf.predict(X_train) == y_train).mean()
val_acc = (clf.predict(X_test) == y_test).mean()
print('Training set accuracy: {:.2f}% / validation set: {:.2f}%'.format(100*train_acc, 100*val_acc))